## 🚀 Deep Scraper for PE Dashboard Data

### Lab 1 Enhanced - Comprehensive Data Collection

This scraper implements **deep scraping** to extract ALL data needed for the PE Dashboard payload directly from company websites.

### Features:
1. **📚 12 Page Types** - Homepage, About, Product, Careers, Blog, Team, Investors, Customers, Press, Pricing, Partners, Contact
2. **📝 Blog Post Extraction** - Discovers and scrapes individual blog posts (up to 20 per company) for funding announcements and events
3. **🔍 Smart Page Discovery** - Analyzes homepage links when direct URL patterns fail
4. **📊 Structured Parsing** - Extracts structured data (HQ, founding year, team members, investors, pricing tiers, etc.)
5. **🚫 Bypasses robots.txt** - Gets complete data (for academic use)
6. **🚀 HTTP-First** - Fast and reliable (Playwright available as fallback)

### Data Extraction:
- **Footer/Contact** → HQ city, state, country, founding year
- **Team Page** → Leadership names, roles, bios, LinkedIn
- **Investors Page** → Funding rounds, investor names, amounts
- **Customers Page** → Reference customer names
- **Pricing Page** → Pricing model, tiers, features
- **Partners Page** → Integration partner names
- **Blog Posts** → Individual posts for event extraction (funding, leadership changes, product launches)

### Expected Results:
- **20-40 pages per company** (12 main pages + 10-20 blog posts)
- **4-8 structured JSON files** with parsed data
- **80-95% dashboard field coverage** from scraped data alone

### Configuration:
- **Current Mode**: HTTP + Deep Scraping + No robots.txt
- **To Change**: Modify Cell 10 parameters:
  - `force_playwright=True` - Use Playwright (for JS-heavy sites)
  - `respect_robots=True` - Honor robots.txt


# Deep Forbes AI50 Web Scraper - Lab 1 Enhanced

**Production-ready deep scraper for PE Dashboard data collection**

Features:
- 12 page types + individual blog posts (20-40 pages per company)
- Structured data parsing (HQ, team, investors, pricing, customers, partners)
- Smart page discovery with homepage link analysis
- HTTP-first with Playwright fallback
- Bypasses robots.txt for comprehensive data (academic use)
- Zero hardcoded values - fully dynamic

In [13]:
# Setup and Imports
import json
import time
import nest_asyncio
from pathlib import Path
from datetime import datetime, timezone
from urllib.parse import urljoin, urlparse
from urllib.robotparser import RobotFileParser
from typing import Dict, List, Optional, Tuple

import requests
from bs4 import BeautifulSoup
import trafilatura
from playwright.sync_api import sync_playwright, TimeoutError as PlaywrightTimeout

# Fix Playwright in Jupyter
nest_asyncio.apply()

# Configuration
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
SEED_FILE = PROJECT_ROOT / "data/forbes_ai50_seed.json"
OUTPUT_DIR = PROJECT_ROOT / "data/raw"

# Scraper settings
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"
REQUEST_DELAY = 2
REQUEST_TIMEOUT = 10
MAX_RETRIES = 3
PLAYWRIGHT_TIMEOUT = 15000
SCRAPER_VERSION = "2.0"

# Page patterns - Expanded for comprehensive scraping
PAGE_PATTERNS = {
    "homepage": ["/"],
    "about": ["/about", "/company", "/about-us", "/who-we-are", "/our-story"],
    "product": ["/product", "/products", "/platform", "/solutions", "/features"],
    "careers": ["/careers", "/jobs", "/join-us", "/work-with-us"],
    "blog": ["/blog", "/news", "/press", "/newsroom", "/insights", "/resources"],
    # New page types for comprehensive dashboard data
    "team": ["/team", "/leadership", "/about/team", "/about/leadership", "/people", "/our-team"],
    "investors": ["/investors", "/funding", "/about/investors", "/backed-by", "/backers"],
    "customers": ["/customers", "/case-studies", "/success-stories", "/testimonials", "/customer-stories"],
    "press": ["/press", "/newsroom", "/media", "/news-and-press", "/press-releases"],
    "pricing": ["/pricing", "/plans", "/price", "/buy", "/purchase"],
    "partners": ["/partners", "/integrations", "/ecosystem", "/partner", "/integration"],
    "contact": ["/contact", "/contact-us", "/get-in-touch", "/reach-us"]
}

# Track request times for rate limiting
last_request_time = {}

print("✓ Setup complete")
print(f"✓ Project root: {PROJECT_ROOT}")
print(f"✓ Playwright fixed for Jupyter with nest_asyncio")

✓ Setup complete
✓ Project root: /Users/RiyanshiKedia/Documents/GitHub/project_orbit
✓ Playwright fixed for Jupyter with nest_asyncio


In [14]:
# Enhanced Page Discovery Function - Now supports all 12 page types

def discover_links_from_homepage(homepage_html: str, base_url: str) -> Dict[str, str]:
    """Discover page URLs by analyzing homepage links for all page types"""
    discovered = {}
    try:
        soup = BeautifulSoup(homepage_html, 'lxml')
        parsed_base = urlparse(base_url)
        
        # Get all links
        for link in soup.find_all('a', href=True):
            href = link['href'].lower()
            full_url = urljoin(base_url, link['href'])
            
            # Only consider links from same domain
            if urlparse(full_url).netloc != parsed_base.netloc:
                continue
            
            link_text = link.get_text().lower().strip()
            
            # Match all 12 page types based on URL or link text
            
            # About
            if not discovered.get('about'):
                if any(x in href for x in ['/about', '/company', '/who-we-are', '/our-story']):
                    discovered['about'] = full_url
                elif any(x in link_text for x in ['about', 'company', 'who we are', 'our story']):
                    discovered['about'] = full_url
            
            # Product
            if not discovered.get('product'):
                if any(x in href for x in ['/product', '/platform', '/solution', '/feature']):
                    discovered['product'] = full_url
                elif any(x in link_text for x in ['product', 'platform', 'solution', 'features']):
                    discovered['product'] = full_url
            
            # Careers
            if not discovered.get('careers'):
                if any(x in href for x in ['/career', '/job', '/join', '/work-with']):
                    discovered['careers'] = full_url
                elif any(x in link_text for x in ['career', 'jobs', 'join us', 'work with']):
                    discovered['careers'] = full_url
            
            # Blog
            if not discovered.get('blog'):
                if any(x in href for x in ['/blog', '/insight', '/resource']):
                    discovered['blog'] = full_url
                elif any(x in link_text for x in ['blog', 'insights', 'resources']):
                    discovered['blog'] = full_url
            
            # Team
            if not discovered.get('team'):
                if any(x in href for x in ['/team', '/leadership', '/people', '/our-team']):
                    discovered['team'] = full_url
                elif any(x in link_text for x in ['team', 'leadership', 'people', 'our team']):
                    discovered['team'] = full_url
            
            # Investors
            if not discovered.get('investors'):
                if any(x in href for x in ['/investor', '/funding', '/backed-by', '/backer']):
                    discovered['investors'] = full_url
                elif any(x in link_text for x in ['investors', 'funding', 'backed by', 'backers']):
                    discovered['investors'] = full_url
            
            # Customers
            if not discovered.get('customers'):
                if any(x in href for x in ['/customer', '/case-stud', '/success-stor', '/testimonial']):
                    discovered['customers'] = full_url
                elif any(x in link_text for x in ['customers', 'case studies', 'success stories', 'testimonials']):
                    discovered['customers'] = full_url
            
            # Press
            if not discovered.get('press'):
                if any(x in href for x in ['/press', '/newsroom', '/media', '/news-and-press']):
                    discovered['press'] = full_url
                elif any(x in link_text for x in ['press', 'newsroom', 'media', 'news']):
                    discovered['press'] = full_url
            
            # Pricing
            if not discovered.get('pricing'):
                if any(x in href for x in ['/pricing', '/plans', '/price', '/buy']):
                    discovered['pricing'] = full_url
                elif any(x in link_text for x in ['pricing', 'plans', 'price', 'buy']):
                    discovered['pricing'] = full_url
            
            # Partners
            if not discovered.get('partners'):
                if any(x in href for x in ['/partner', '/integration', '/ecosystem']):
                    discovered['partners'] = full_url
                elif any(x in link_text for x in ['partners', 'integrations', 'ecosystem']):
                    discovered['partners'] = full_url
            
            # Contact
            if not discovered.get('contact'):
                if any(x in href for x in ['/contact', '/get-in-touch', '/reach-us']):
                    discovered['contact'] = full_url
                elif any(x in link_text for x in ['contact', 'get in touch', 'reach us']):
                    discovered['contact'] = full_url
    
    except Exception as e:
        print(f"      Warning: Link discovery failed - {str(e)[:50]}")
    
    return discovered

print("✓ Enhanced page discovery loaded (supports all 12 page types)")


✓ Enhanced page discovery loaded (supports all 12 page types)


In [15]:
# Core Helper Functions

def check_robots_txt(base_url: str) -> bool:
    """Check if scraping is allowed by robots.txt"""
    try:
        parsed = urlparse(base_url)
        robots_url = f"{parsed.scheme}://{parsed.netloc}/robots.txt"
        rp = RobotFileParser()
        rp.set_url(robots_url)
        rp.read()
        return rp.can_fetch(USER_AGENT, base_url)
    except:
        return True  # Assume allowed if can't read


def find_page_url(base_url: str, page_type: str) -> Optional[str]:
    """Find URL for a page type by trying multiple patterns"""
    patterns = PAGE_PATTERNS.get(page_type, [])
    for pattern in patterns:
        url = urljoin(base_url, pattern)
        try:
            response = requests.head(url, timeout=5, headers={"User-Agent": USER_AGENT}, allow_redirects=True)
            if response.status_code == 200:
                return response.url
        except:
            continue
    return None


def extract_clean_text(html: str) -> str:
    """Extract clean text from HTML"""
    try:
        clean_text = trafilatura.extract(html, include_comments=False, include_tables=True)
        if clean_text and len(clean_text) > 100:
            return clean_text
    except:
        pass
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        for element in soup(['script', 'style', 'nav', 'footer', 'header']):
            element.decompose()
        text = soup.get_text(separator='\n', strip=True)
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        return '\n'.join(lines)
    except:
        return ""


def save_page_data(company_dir: Path, page_type: str, html: str, clean_text: str) -> Dict[str, int]:
    """Save raw HTML and clean text"""
    html_path = company_dir / f"{page_type}.html"
    txt_path = company_dir / f"{page_type}_clean.txt"
    
    html_path.write_text(html, encoding='utf-8')
    txt_path.write_text(clean_text, encoding='utf-8')
    
    return {"html_size": len(html), "text_size": len(clean_text)}


print("✓ Helper functions loaded")


✓ Helper functions loaded


In [16]:
# Blog Post Extractor

import re
import hashlib

def extract_blog_post_links(blog_html: str, base_url: str, limit: int = 20) -> List[str]:
    """
    Extract individual blog post URLs from blog index page.
    Returns up to 'limit' most recent posts.
    """
    discovered_posts = []
    seen_urls = set()
    
    try:
        soup = BeautifulSoup(blog_html, 'lxml')
        parsed_base = urlparse(base_url)
        
        # Common blog post selectors
        post_selectors = [
            'article a',
            '.post a',
            '.blog-post a',
            '.entry a',
            '[class*="article"] a',
            '[class*="post"] a',
            'h2 a',
            'h3 a'
        ]
        
        # Try each selector
        for selector in post_selectors:
            links = soup.select(selector)
            for link in links:
                if not link.get('href'):
                    continue
                    
                full_url = urljoin(base_url, link['href'])
                parsed_url = urlparse(full_url)
                
                # Only same domain
                if parsed_url.netloc != parsed_base.netloc:
                    continue
                
                # Skip common non-post pages
                skip_patterns = [
                    '/category/', '/tag/', '/author/', '/page/',
                    '/search', '/archive', '#', 'javascript:',
                    '.pdf', '.jpg', '.png', '.gif'
                ]
                if any(pattern in full_url.lower() for pattern in skip_patterns):
                    continue
                
                # Avoid duplicates
                if full_url in seen_urls:
                    continue
                    
                # Look for date patterns or post IDs in URL
                # This helps identify actual blog posts
                if any(pattern in full_url for pattern in ['/blog/', '/news/', '/post/', '/article/', '/20']):
                    seen_urls.add(full_url)
                    discovered_posts.append(full_url)
                    
                    if len(discovered_posts) >= limit:
                        break
            
            if len(discovered_posts) >= limit:
                break
        
        # If we didn't find enough, try all links that contain the blog path
        if len(discovered_posts) < 5:
            all_links = soup.find_all('a', href=True)
            for link in all_links:
                full_url = urljoin(base_url, link['href'])
                parsed_url = urlparse(full_url)
                
                if parsed_url.netloc != parsed_base.netloc:
                    continue
                    
                path = parsed_url.path.lower()
                if ('/blog/' in path or '/news/' in path) and full_url not in seen_urls:
                    skip_patterns = ['/category/', '/tag/', '/author/', '/page/', '/search', '/archive']
                    if not any(pattern in path for pattern in skip_patterns):
                        discovered_posts.append(full_url)
                        seen_urls.add(full_url)
                        
                        if len(discovered_posts) >= limit:
                            break
    
    except Exception as e:
        print(f"      Warning: Blog post extraction failed - {str(e)[:50]}")
    
    return discovered_posts[:limit]


print("✓ Blog post extractor loaded")


✓ Blog post extractor loaded


In [17]:
# Structured Page Parsers

def parse_footer(html: str) -> Dict:
    """Extract HQ location and founding year from footer/about page"""
    result = {
        "hq_city": None,
        "hq_state": None,
        "hq_country": None,
        "founded_year": None
    }
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        text = soup.get_text().lower()
        
        # Find founding year (look for copyright or "founded in" patterns)
        year_patterns = [
            r'©\s*(\d{4})',
            r'copyright\s*(\d{4})',
            r'founded\s+in\s+(\d{4})',
            r'established\s+in\s+(\d{4})',
            r'since\s+(\d{4})'
        ]
        for pattern in year_patterns:
            match = re.search(pattern, text)
            if match:
                year = int(match.group(1))
                if 2000 <= year <= 2025:  # Reasonable range for AI companies
                    result["founded_year"] = year
                    break
        
        # Find HQ (look for address patterns, common cities)
        # Common AI hubs
        us_cities = {
            'san francisco': ('San Francisco', 'CA', 'US'),
            'palo alto': ('Palo Alto', 'CA', 'US'),
            'mountain view': ('Mountain View', 'CA', 'US'),
            'new york': ('New York', 'NY', 'US'),
            'seattle': ('Seattle', 'WA', 'US'),
            'boston': ('Boston', 'MA', 'US'),
            'cambridge': ('Cambridge', 'MA', 'US'),
            'austin': ('Austin', 'TX', 'US')
        }
        
        for city_key, (city, state, country) in us_cities.items():
            if city_key in text:
                result["hq_city"] = city
                result["hq_state"] = state
                result["hq_country"] = country
                break
        
        # Try to find address tags
        address_tag = soup.find('address')
        if address_tag:
            address_text = address_tag.get_text()
            # Try to extract city/state/country from structured address
            for city_key, (city, state, country) in us_cities.items():
                if city_key in address_text.lower():
                    result["hq_city"] = city
                    result["hq_state"] = state
                    result["hq_country"] = country
                    break
        
    except Exception as e:
        pass
    
    return result


def parse_team_page(html: str) -> List[Dict]:
    """Extract team member information"""
    team_members = []
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        
        # Common team member containers
        member_selectors = [
            '.team-member',
            '.person',
            '.employee',
            '[class*="team"]',
            '[class*="member"]',
            'article'
        ]
        
        for selector in member_selectors:
            members = soup.select(selector)
            if len(members) > 1:  # Found a pattern
                for member in members[:20]:  # Limit to top 20
                    member_data = {
                        "name": None,
                        "role": None,
                        "bio": None,
                        "linkedin": None
                    }
                    
                    # Extract name (usually in h2, h3, or strong)
                    name_tag = member.find(['h2', 'h3', 'h4', 'strong'])
                    if name_tag:
                        member_data["name"] = name_tag.get_text().strip()
                    
                    # Extract role/title
                    role_classes = ['role', 'title', 'position', 'job-title']
                    for cls in role_classes:
                        role_tag = member.find(class_=lambda x: x and cls in x.lower())
                        if role_tag:
                            member_data["role"] = role_tag.get_text().strip()
                            break
                    
                    # If no role found, try p tags
                    if not member_data["role"]:
                        p_tags = member.find_all('p')
                        if len(p_tags) > 0:
                            first_p = p_tags[0].get_text().strip()
                            if len(first_p) < 100:  # Likely a title, not bio
                                member_data["role"] = first_p
                    
                    # Extract bio
                    bio_tag = member.find('p', class_=lambda x: x and 'bio' in x.lower() if x else False)
                    if bio_tag:
                        member_data["bio"] = bio_tag.get_text().strip()
                    
                    # Extract LinkedIn
                    linkedin_link = member.find('a', href=lambda x: x and 'linkedin.com' in x if x else False)
                    if linkedin_link:
                        member_data["linkedin"] = linkedin_link['href']
                    
                    if member_data["name"]:  # Only add if we got a name
                        team_members.append(member_data)
                
                if team_members:  # If we found members, stop trying selectors
                    break
    
    except Exception as e:
        pass
    
    return team_members


def parse_investors_page(html: str) -> List[Dict]:
    """Extract investor and funding information"""
    investors_data = []
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        text = soup.get_text()
        
        # Look for funding round mentions
        funding_patterns = [
            r'(seed|series [a-z]|series [0-9])\s+round',
            r'raised\s+\$?([\d.]+)\s*(million|billion|m|b)',
            r'\$?([\d.]+)\s*(million|billion|m|b)\s+in\s+funding'
        ]
        
        for pattern in funding_patterns:
            matches = re.finditer(pattern, text.lower())
            for match in matches:
                investors_data.append({
                    "snippet": match.group(0),
                    "type": "funding_mention"
                })
        
        # Extract investor names (usually in lists or with logos)
        investor_containers = soup.find_all(['ul', 'div'], class_=lambda x: x and ('investor' in x.lower() or 'backer' in x.lower()) if x else False)
        for container in investor_containers:
            items = container.find_all(['li', 'div'])
            for item in items:
                investor_name = item.get_text().strip()
                if investor_name and len(investor_name) < 100:
                    investors_data.append({
                        "name": investor_name,
                        "type": "investor"
                    })
    
    except Exception as e:
        pass
    
    return investors_data


def parse_customers_page(html: str) -> List[str]:
    """Extract customer/client names"""
    customers = []
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        
        # Look for customer logos
        customer_imgs = soup.find_all('img', alt=True)
        for img in customer_imgs:
            alt_text = img.get('alt', '').strip()
            # Filter out likely non-customer images
            if alt_text and len(alt_text) < 100 and 'logo' not in alt_text.lower():
                customers.append(alt_text)
        
        # Look for customer lists
        customer_sections = soup.find_all(['ul', 'div'], class_=lambda x: x and ('customer' in x.lower() or 'client' in x.lower()) if x else False)
        for section in customer_sections:
            items = section.find_all(['li', 'div'])
            for item in items:
                customer_name = item.get_text().strip()
                if customer_name and len(customer_name) < 100:
                    customers.append(customer_name)
    
    except Exception as e:
        pass
    
    return list(set(customers))[:50]  # Deduplicate and limit


def parse_pricing_page(html: str) -> Dict:
    """Extract pricing information"""
    pricing_data = {
        "pricing_model": None,
        "tiers": []
    }
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        text = soup.get_text().lower()
        
        # Detect pricing model
        if 'per seat' in text or 'per user' in text:
            pricing_data["pricing_model"] = "per-seat"
        elif 'usage-based' in text or 'pay as you go' in text:
            pricing_data["pricing_model"] = "usage-based"
        elif 'enterprise' in text and 'contact' in text:
            pricing_data["pricing_model"] = "enterprise"
        
        # Extract tier names
        tier_patterns = ['free', 'starter', 'basic', 'pro', 'professional', 'business', 'enterprise', 'premium', 'plus']
        
        # Look for pricing cards/sections
        pricing_cards = soup.find_all(['div', 'section'], class_=lambda x: x and ('price' in x.lower() or 'tier' in x.lower() or 'plan' in x.lower()) if x else False)
        
        for card in pricing_cards:
            card_text = card.get_text().lower()
            for tier_name in tier_patterns:
                if tier_name in card_text:
                    # Try to find price
                    price_match = re.search(r'\$\s*(\d+(?:,\d{3})*(?:\.\d{2})?)', card.get_text())
                    price = price_match.group(0) if price_match else None
                    
                    pricing_data["tiers"].append({
                        "name": tier_name.capitalize(),
                        "price": price
                    })
                    break
        
        # If no tiers found, look for tier names in headings
        if not pricing_data["tiers"]:
            headings = soup.find_all(['h2', 'h3', 'h4'])
            for heading in headings:
                heading_text = heading.get_text().lower()
                for tier_name in tier_patterns:
                    if tier_name in heading_text:
                        pricing_data["tiers"].append({
                            "name": tier_name.capitalize(),
                            "price": None
                        })
                        break
    
    except Exception as e:
        pass
    
    return pricing_data


def parse_partners_page(html: str) -> List[str]:
    """Extract integration partner names"""
    partners = []
    
    try:
        soup = BeautifulSoup(html, 'lxml')
        
        # Look for partner logos with alt text
        partner_imgs = soup.find_all('img', alt=True)
        for img in partner_imgs:
            alt_text = img.get('alt', '').strip()
            if alt_text and len(alt_text) < 100:
                partners.append(alt_text)
        
        # Look for partner lists
        partner_sections = soup.find_all(['ul', 'div'], class_=lambda x: x and ('partner' in x.lower() or 'integration' in x.lower()) if x else False)
        for section in partner_sections:
            items = section.find_all(['li', 'a'])
            for item in items:
                partner_name = item.get_text().strip()
                if partner_name and len(partner_name) < 100:
                    partners.append(partner_name)
    
    except Exception as e:
        pass
    
    return list(set(partners))[:50]  # Deduplicate and limit


print("✓ Structured parsers loaded (6 functions)")


✓ Structured parsers loaded (6 functions)


In [18]:
# Fetching Functions with Smart Fallback

def fetch_with_requests(url: str) -> Tuple[Optional[str], int, str]:
    """Fetch page with requests library (fast)"""
    domain = urlparse(url).netloc
    
    # Rate limiting
    if domain in last_request_time:
        elapsed = time.time() - last_request_time[domain]
        if elapsed < REQUEST_DELAY:
            time.sleep(REQUEST_DELAY - elapsed)
    
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.get(url, timeout=REQUEST_TIMEOUT, headers={"User-Agent": USER_AGENT}, allow_redirects=True)
            last_request_time[domain] = time.time()
            
            if response.status_code == 200:
                return response.text, 200, "HTTP Success"
            elif response.status_code == 404:
                return None, 404, "Not found"
            elif response.status_code in [403, 429]:
                wait_time = 2 ** attempt
                time.sleep(wait_time)
                continue
            else:
                return None, response.status_code, f"HTTP {response.status_code}"
        except requests.Timeout:
            if attempt < MAX_RETRIES - 1:
                time.sleep(2 ** attempt)
            else:
                return None, 0, "Timeout"
        except Exception as e:
            return None, 0, f"Error: {str(e)[:50]}"
    
    return None, 0, "Max retries"


def fetch_with_playwright(url: str) -> Tuple[Optional[str], int, str]:
    """Fetch page with Playwright (handles JS, blocking)"""
    try:
        with sync_playwright() as p:
            browser = p.chromium.launch(headless=True)
            context = browser.new_context(
                user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
                viewport={"width": 1920, "height": 1080}
            )
            page = context.new_page()
            
            try:
                response = page.goto(url, timeout=PLAYWRIGHT_TIMEOUT, wait_until="networkidle")
                page.wait_for_timeout(2000)
                html = page.content()
                status_code = response.status if response else 200
                browser.close()
                
                if status_code == 200:
                    return html, 200, "Playwright Success"
                else:
                    return None, status_code, f"Playwright HTTP {status_code}"
            except PlaywrightTimeout:
                browser.close()
                return None, 0, "Playwright timeout"
            except Exception as e:
                browser.close()
                return None, 0, f"Playwright error: {str(e)[:50]}"
    except Exception as e:
        return None, 0, f"Playwright init failed: {str(e)[:50]}"


def fetch_page_smart(url: str, force_playwright: bool = False) -> Tuple[Optional[str], int, str]:
    """Smart fetch with automatic fallback"""
    if force_playwright:
        return fetch_with_playwright(url)
    
    # Try HTTP first (fast)
    html, status_code, note = fetch_with_requests(url)
    
    # If blocked/failed, try Playwright
    if html is None and status_code in [0, 403, 429]:
        print("      → Fallback to Playwright")
        return fetch_with_playwright(url)
    
    return html, status_code, note


print("✓ Fetch functions with smart fallback loaded")


✓ Fetch functions with smart fallback loaded


In [19]:
# Main Scraper Function - Enhanced for Deep Scraping

def scrape_company(company: Dict, force_playwright: bool = False, respect_robots: bool = True) -> Dict:
    """
    Deep scrape all pages for a company with smart fallbacks and structured parsing.
    
    Args:
        company: Company dict with name, website, company_id
        force_playwright: If True, use Playwright for all pages
        respect_robots: If False, bypass robots.txt (use ethically!)
    """
    company_name = company["company_name"]
    company_id = company["company_id"]
    base_url = company["website"]
    
    print(f"\n{'='*70}")
    print(f"🔍 {company_name} ({company_id})")
    print(f"   {base_url}")
    print(f"   Mode: {'🎭 Playwright' if force_playwright else '🚀 HTTP + Playwright fallback'}")
    print(f"{'='*70}")
    
    # Create folders
    company_dir = OUTPUT_DIR / company_id / "initial_pull"
    blog_posts_dir = company_dir / "blog_posts"
    company_dir.mkdir(parents=True, exist_ok=True)
    blog_posts_dir.mkdir(parents=True, exist_ok=True)
    
    # Check robots.txt
    if respect_robots and not force_playwright:
        if not check_robots_txt(base_url):
            print("   ✗ Blocked by robots.txt (set respect_robots=False to bypass)")
            return {
                "company_name": company_name,
                "company_id": company_id,
                "status": "blocked_by_robots",
                "pages_scraped": 0,
                "pages_total": 12
            }
    
    # Scrape main pages (12 types)
    page_results = []
    pages_scraped = 0
    homepage_html = None
    discovered_links = {}
    all_page_types = ["homepage", "about", "product", "careers", "blog", "team", "investors", "customers", "press", "pricing", "partners", "contact"]
    
    for page_type in all_page_types:
        print(f"   📄 {page_type.capitalize()}...", end=" ")
        
        # Find URL
        if page_type == "homepage":
            page_url = base_url
        else:
            # Try predefined patterns first
            page_url = find_page_url(base_url, page_type)
            
            # If not found and we have homepage HTML, try discovering from links
            if not page_url and homepage_html:
                if not discovered_links:
                    discovered_links = discover_links_from_homepage(homepage_html, base_url)
                page_url = discovered_links.get(page_type)
        
        if not page_url:
            print("Not found")
            page_results.append({
                "page_type": page_type,
                "source_url": None,
                "crawled_at": datetime.now(timezone.utc).isoformat(),
                "status_code": 0,
                "found": False,
                "note": "URL not found"
            })
            continue
        
        # Fetch page
        html, status_code, note = fetch_page_smart(page_url, force_playwright)
        
        # Save homepage HTML for link discovery
        if page_type == "homepage" and html:
            homepage_html = html
        
        if html:
            clean_text = extract_clean_text(html)
            sizes = save_page_data(company_dir, page_type, html, clean_text)
            print(f"✓ ({sizes['html_size']:,}B / {sizes['text_size']:,}B)")
            
            # Parse structured data for specific page types
            structured_data = None
            if page_type in ["about", "contact"]:
                structured_data = parse_footer(html)
            elif page_type == "team":
                structured_data = parse_team_page(html)
            elif page_type == "investors":
                structured_data = parse_investors_page(html)
            elif page_type == "customers":
                structured_data = parse_customers_page(html)
            elif page_type == "pricing":
                structured_data = parse_pricing_page(html)
            elif page_type == "partners":
                structured_data = parse_partners_page(html)
            elif page_type == "careers":
                # For careers, we could extract job openings
                structured_data = {"note": "Job openings can be parsed in Lab 5"}
            
            # Save structured data if we extracted any
            if structured_data:
                structured_path = company_dir / f"{page_type}_structured.json"
                structured_path.write_text(json.dumps(structured_data, indent=2), encoding='utf-8')
            
            page_results.append({
                "page_type": page_type,
                "source_url": page_url,
                "crawled_at": datetime.now(timezone.utc).isoformat(),
                "status_code": status_code,
                "content_length": sizes['html_size'],
                "found": True,
                "note": note,
                "method": "playwright" if "Playwright" in note else "http",
                "has_structured_data": structured_data is not None
            })
            pages_scraped += 1
            
            # Extract and scrape blog posts if this is the blog page
            if page_type == "blog" and html:
                print(f"\n      🔍 Extracting blog posts...", end=" ")
                blog_post_urls = extract_blog_post_links(html, base_url, limit=20)
                print(f"Found {len(blog_post_urls)} posts")
                
                for i, post_url in enumerate(blog_post_urls):
                    # Create short hash for filename
                    url_hash = hashlib.md5(post_url.encode()).hexdigest()[:8]
                    post_filename = f"post_{url_hash}"
                    
                    # Fetch blog post
                    post_html, post_status, post_note = fetch_page_smart(post_url, force_playwright)
                    
                    if post_html:
                        post_clean_text = extract_clean_text(post_html)
                        
                        # Save to blog_posts subfolder
                        (blog_posts_dir / f"{post_filename}.html").write_text(post_html, encoding='utf-8')
                        (blog_posts_dir / f"{post_filename}_clean.txt").write_text(post_clean_text, encoding='utf-8')
                        
                        pages_scraped += 1
                        
                        if (i + 1) % 5 == 0:
                            print(f"      ... {i+1}/{len(blog_post_urls)} blog posts scraped")
                
                if len(blog_post_urls) > 0:
                    print(f"      ✓ {len(blog_post_urls)} blog posts saved to blog_posts/")
        else:
            print(f"✗ {note}")
            page_results.append({
                "page_type": page_type,
                "source_url": page_url,
                "crawled_at": datetime.now(timezone.utc).isoformat(),
                "status_code": status_code,
                "found": False,
                "note": note
            })
    
    # Save metadata
    metadata = {
        "company_name": company_name,
        "company_id": company_id,
        "scrape_timestamp": datetime.now(timezone.utc).isoformat(),
        "scraper_version": "3.0-deep",
        "force_playwright": force_playwright,
        "respect_robots": respect_robots,
        "pages": page_results
    }
    
    metadata_path = company_dir / "metadata.json"
    metadata_path.write_text(json.dumps(metadata, indent=2), encoding='utf-8')
    
    print(f"   ✅ {pages_scraped} total pages/posts scraped")
    
    return {
        "company_name": company_name,
        "company_id": company_id,
        "status": "success" if pages_scraped > 0 else "failed",
        "pages_scraped": pages_scraped,
        "pages_total": 12,
        "blog_posts_found": len([p for p in page_results if p.get("page_type") == "blog" and p.get("found")])
    }


print("✓ Main scraper loaded (deep scraping with 12 page types + blog posts)")


✓ Main scraper loaded (deep scraping with 12 page types + blog posts)


In [20]:
# Load Companies

with open(SEED_FILE, 'r') as f:
    all_companies = json.load(f)

# Add company_id to all
for company in all_companies:
    domain = urlparse(company["website"]).netloc
    company["company_id"] = domain.replace("www.", "").split(".")[0]

print(f"✓ Loaded {len(all_companies)} companies from Forbes AI 50")
print(f"✓ Ready to deep scrape ALL {len(all_companies)} companies")
print(f"\nFirst 5 companies (preview):")
for c in all_companies[:5]:
    print(f"  • {c['company_name']:20} ({c['company_id']:15}) - {c['website']}")


✓ Loaded 50 companies from Forbes AI 50
✓ Ready to deep scrape ALL 50 companies

First 5 companies (preview):
  • Abridge              (abridge        ) - https://www.abridge.com
  • Anthropic            (anthropic      ) - https://www.anthropic.com
  • Anysphere            (cursor         ) - https://www.cursor.com
  • Baseten              (baseten        ) - https://www.baseten.co
  • Captions             (captions       ) - https://www.captions.ai

⚠️  Note: Full scrape will take ~2-4 hours for all 50 companies


In [21]:
# Scrape ALL 50 Companies (Deep Scraping)
# This will:
# 1. Use HTTP requests (fast and reliable)
# 2. Bypass robots.txt to get complete data
# 3. Scrape 12 page types + up to 20 blog posts per company
# 4. Extract structured data (HQ, team, investors, pricing, etc.)
# 5. Save all data to data/raw/<company_id>/initial_pull/

import warnings
warnings.filterwarnings('ignore')

# Use ALL companies (not just test subset)
companies_to_scrape = all_companies

results = []
start_time = time.time()

print(f"\n{'='*70}")
print(f"🚀 DEEP SCRAPING ALL {len(companies_to_scrape)} FORBES AI50 COMPANIES")
print(f"   Mode: 🚀 HTTP + Deep Scraping (bypassing robots.txt)")
print(f"   Pages: 12 page types + up to 20 blog posts each")
print(f"{'='*70}\n")

for i, company in enumerate(companies_to_scrape, 1):
    print(f"[{i}/{len(companies_to_scrape)}] {company['company_name']}...")
    
    try:
        # Use HTTP (not Playwright) but don't respect robots.txt
        result = scrape_company(company, force_playwright=False, respect_robots=False)
        results.append(result)
    except Exception as e:
        print(f"   ✗ ERROR: {e}")
        results.append({
            "company_name": company['company_name'],
            "company_id": company['company_id'],
            "status": "error",
            "error": str(e),
            "pages_scraped": 0,
            "pages_total": 12
        })
    
    # Progress update every 5 companies
    if i % 5 == 0 or i == len(companies_to_scrape):
        elapsed = time.time() - start_time
        avg_time_per_company = elapsed / i
        remaining = (len(companies_to_scrape) - i) * avg_time_per_company
        successful_so_far = sum(1 for r in results if r.get('status') == 'success')
        print(f"\n   ⏱  Progress: {i}/{len(companies_to_scrape)} companies | {successful_so_far} successful | {elapsed/60:.1f}m elapsed | ~{remaining/60:.1f}m remaining\n")

# Final Summary
elapsed = time.time() - start_time
successful = [r for r in results if r.get('status') == 'success']
blocked = [r for r in results if r.get('status') == 'blocked_by_robots']
failed = [r for r in results if r.get('status') not in ['success', 'blocked_by_robots']]
total_pages = sum(r.get('pages_scraped', 0) for r in results)
avg_pages = total_pages / len(companies_to_scrape) if companies_to_scrape else 0

print(f"\n{'='*70}")
print(f"🎉 DEEP SCRAPING COMPLETE - ALL {len(companies_to_scrape)} COMPANIES")
print(f"{'='*70}")
print(f"✓ Successful:            {len(successful)}/{len(companies_to_scrape)} companies ({len(successful)/len(companies_to_scrape)*100:.1f}%)")
print(f"🚫 Blocked (robots):      {len(blocked)} companies")
print(f"✗ Failed:                {len(failed)} companies")
print(f"📄 Total pages/posts:    {total_pages:,}")
print(f"📊 Average per company:  {avg_pages:.1f} pages/posts")
print(f"⏱  Total time:           {elapsed/60:.1f} minutes ({elapsed/len(companies_to_scrape):.1f}s per company)")
print(f"{'='*70}")

# Show blocked companies (should be none)
if blocked:
    print(f"\n🚫 Still blocked:")
    for r in blocked:
        print(f"   • {r['company_name']}")

# Show failed companies
if failed:
    print(f"\n✗ Failed companies:")
    for r in failed:
        print(f"   • {r['company_name']}: {r.get('error', 'Unknown error')}")
else:
    print(f"\n✅ All companies scraped successfully!")
    print(f"✅ Expected: 20-40 pages per company (12 main + 10-20 blog posts)")



🚀 DEEP SCRAPING ALL 50 FORBES AI50 COMPANIES
   Mode: 🚀 HTTP + Deep Scraping (bypassing robots.txt)
   Pages: 12 page types + up to 20 blog posts each
   Expected time: ~2-4 hours for all companies

[1/50] Abridge...

🔍 Abridge (abridge)
   https://www.abridge.com
   Mode: 🚀 HTTP + Playwright fallback
   📄 Homepage... ✓ (197,005B / 562B)
   📄 About... ✓ (44,949B / 2,159B)
   📄 Product... ✓ (120,057B / 1,491B)
   📄 Careers... ✓ (87,122B / 59B)
   📄 Blog... ✓ (37,718B / 874B)

      🔍 Extracting blog posts... Found 4 posts
      ✓ 4 blog posts saved to blog_posts/
   📄 Team... ✓ (44,949B / 2,159B)
   📄 Investors... Not found
   📄 Customers... ✓ (110,137B / 4,361B)
   📄 Press... ✓ (41,595B / 1,520B)
   📄 Pricing... Not found
   📄 Partners... ✓ (33,614B / 5,737B)
   📄 Contact... ✓ (100,087B / 159B)
   ✅ 14 total pages/posts scraped
[2/50] Anthropic...

🔍 Anthropic (anthropic)
   https://www.anthropic.com
   Mode: 🚀 HTTP + Playwright fallback
   📄 Homepage... ✓ (273,364B / 695B)
   📄 About

In [22]:
# Validation Report (Deep Scraping - ALL Companies)

print(f"\n{'='*70}")
print(f"📊 VALIDATION REPORT - DEEP SCRAPING (ALL {len(all_companies)} COMPANIES)")
print(f"{'='*70}\n")

validation_results = []

for company in all_companies:
    company_id = company['company_id']
    company_name = company['company_name']
    company_dir = OUTPUT_DIR / company_id / "initial_pull"
    
    if not company_dir.exists():
        print(f"✗ {company_name:25} - Folder missing")
        validation_results.append({
            "company": company_name, 
            "status": "missing", 
            "pages": 0,
            "structured_files": 0,
            "blog_posts": 0
        })
        continue
    
    metadata_path = company_dir / "metadata.json"
    if metadata_path.exists():
        metadata = json.loads(metadata_path.read_text())
        pages_found = sum(1 for p in metadata['pages'] if p['found'])
        
        # Count structured JSON files
        structured_files = len(list(company_dir.glob("*_structured.json")))
        
        # Count blog posts
        blog_posts_dir = company_dir / "blog_posts"
        blog_posts_count = 0
        if blog_posts_dir.exists():
            blog_posts_count = len(list(blog_posts_dir.glob("post_*.html")))
        
        total_items = pages_found + blog_posts_count
        status_symbol = '✓' if pages_found >= 8 else '⚠' if pages_found > 0 else '✗'
        
        print(f"{status_symbol} {company_name:25} - {pages_found}/12 pages | {blog_posts_count} blog posts | {structured_files} structured")
        
        validation_results.append({
            "company": company_name, 
            "status": "ok", 
            "pages": pages_found,
            "structured_files": structured_files,
            "blog_posts": blog_posts_count,
            "total_items": total_items
        })
    else:
        print(f"⚠ {company_name:25} - No metadata")
        validation_results.append({
            "company": company_name, 
            "status": "no_metadata", 
            "pages": 0,
            "structured_files": 0,
            "blog_posts": 0
        })

# Summary stats
total_with_data = sum(1 for v in validation_results if v['pages'] > 0)
total_pages_all = sum(v['pages'] for v in validation_results)
total_blog_posts = sum(v.get('blog_posts', 0) for v in validation_results)
total_structured = sum(v.get('structured_files', 0) for v in validation_results)
total_items_all = sum(v.get('total_items', v['pages']) for v in validation_results)

print(f"\n{'='*70}")
print(f"📈 SUMMARY - ALL {len(all_companies)} COMPANIES")
print(f"{'='*70}")
print(f"Companies with data:     {total_with_data}/{len(all_companies)} ({total_with_data/len(all_companies)*100:.1f}%)")
print(f"Total pages scraped:     {total_pages_all:,}/{len(all_companies) * 12} (expected)")
print(f"Total blog posts:        {total_blog_posts:,}")
print(f"Total structured files:  {total_structured:,}")
print(f"Total items (all):       {total_items_all:,}")
print(f"Average pages/company:   {total_pages_all/len(all_companies):.1f}")
print(f"Average items/company:   {total_items_all/len(all_companies):.1f}")
print(f"{'='*70}")
print(f"\n✅ Check data/raw/<company_id>/initial_pull/ for:")
print(f"   - HTML & clean text files for 12 page types")
print(f"   - *_structured.json files for parsed data")
print(f"   - blog_posts/ subfolder with individual posts")
print(f"{'='*70}")



📊 VALIDATION REPORT - DEEP SCRAPING (ALL 50 COMPANIES)

✓ Abridge                   - 10/12 pages | 4 blog posts | 6 structured
✓ Anthropic                 - 11/12 pages | 20 blog posts | 5 structured
✓ Anysphere                 - 9/12 pages | 10 blog posts | 5 structured
✓ Baseten                   - 10/12 pages | 9 blog posts | 6 structured
⚠ Captions                  - 7/12 pages | 0 blog posts | 4 structured
✓ Clay                      - 11/12 pages | 20 blog posts | 6 structured
✓ Coactive AI               - 9/12 pages | 18 blog posts | 4 structured
✓ Codeium                   - 12/12 pages | 1 blog posts | 6 structured
✓ Cohere                    - 10/12 pages | 0 blog posts | 6 structured
✓ Crusoe                    - 9/12 pages | 12 blog posts | 5 structured
✓ Databricks                - 11/12 pages | 20 blog posts | 6 structured
✓ Decagon                   - 8/12 pages | 0 blog posts | 3 structured
✓ DeepL                     - 11/12 pages | 10 blog posts | 6 structured
✓ Ele

In [23]:
# Export Results Summary (Lab 1 Enhanced - Deep Scraping ALL Companies)

# Calculate statistics
total_pages_posts = sum(r.get('pages_scraped', 0) for r in results)
avg_per_company = total_pages_posts / len(all_companies) if all_companies else 0

# Save detailed results to JSON
results_file = PROJECT_ROOT / "scraping_results_lab1_full.json"
results_file.write_text(json.dumps({
    "scrape_date": datetime.now(timezone.utc).isoformat(),
    "scraper_version": "3.0-deep",
    "total_companies": len(all_companies),
    "successful": len([r for r in results if r.get('status') == 'success']),
    "blocked_by_robots": len([r for r in results if r.get('status') == 'blocked_by_robots']),
    "failed": len([r for r in results if r.get('status') not in ['success', 'blocked_by_robots']]),
    "total_pages_posts": total_pages_posts,
    "average_per_company": round(avg_per_company, 1),
    "companies": results
}, indent=2), encoding='utf-8')

print(f"✅ Results exported to: {results_file}")
print(f"\n📊 Quick Stats:")
print(f"   Success rate:        {len([r for r in results if r.get('status') == 'success'])}/{len(all_companies)} ({len([r for r in results if r.get('status') == 'success'])/len(all_companies)*100:.1f}%)")
print(f"   Total pages/posts:   {total_pages_posts:,}")
print(f"   Average per company: {avg_per_company:.1f}")

print(f"\n🎯 Lab 1 Enhanced Checkpoint Complete!")
print(f"   ✓ Deep scraper implemented (12 page types + blog posts)")
print(f"   ✓ Structured data parsers (HQ, team, investors, pricing, etc.)")
print(f"   ✓ Data stored in data/raw/<company_id>/initial_pull/")
print(f"   ✓ Structured JSON files saved (*_structured.json)")
print(f"   ✓ Blog posts saved in blog_posts/ subfolder")
print(f"   ✓ Metadata JSON generated for each company")
print(f"   ✓ Results summary exported")
print(f"\n📁 Data Structure:")
print(f"   data/raw/<company_id>/initial_pull/")
print(f"      ├── homepage.html / homepage_clean.txt")
print(f"      ├── about.html / about_clean.txt / about_structured.json")
print(f"      ├── team.html / team_clean.txt / team_structured.json")
print(f"      ├── investors.html / investors_clean.txt / investors_structured.json")
print(f"      ├── ... (12 page types total)")
print(f"      ├── blog_posts/")
print(f"      │   ├── post_abc123.html / post_abc123_clean.txt")
print(f"      │   └── ... (up to 20 posts)")
print(f"      └── metadata.json")


✅ Results exported to: /Users/RiyanshiKedia/Documents/GitHub/project_orbit/scraping_results_lab1_full.json

📊 Quick Stats:
   Success rate:        46/50 (92.0%)
   Total pages/posts:   827
   Average per company: 16.5

🎯 Lab 1 Enhanced Checkpoint Complete!
   ✓ Deep scraper implemented (12 page types + blog posts)
   ✓ Structured data parsers (HQ, team, investors, pricing, etc.)
   ✓ Data stored in data/raw/<company_id>/initial_pull/
   ✓ Structured JSON files saved (*_structured.json)
   ✓ Blog posts saved in blog_posts/ subfolder
   ✓ Metadata JSON generated for each company
   ✓ Results summary exported

📁 Data Structure:
   data/raw/<company_id>/initial_pull/
      ├── homepage.html / homepage_clean.txt
      ├── about.html / about_clean.txt / about_structured.json
      ├── team.html / team_clean.txt / team_structured.json
      ├── investors.html / investors_clean.txt / investors_structured.json
      ├── ... (12 page types total)
      ├── blog_posts/
      │   ├── post_abc123.h